# Importações e tratamentos

Essa seção apresenta algumas preparações, como importação de bibliotecas utilizadas, configurações para plotagem de gráficos e definição da paleta de cores utilizada.

Independente de qual seção executar, execute essa primeiro.

In [ ]:
import matplotlib as mpl # Plotagem
import matplotlib.pyplot as plt
import numpy as np # Manipulação matricial
import pandas as pd # Manipulação de DataFrames
import seaborn as sns # Auxílio na plotagem
import os # Manipulação de arquivos a nível de sistema operacional

In [ ]:
mpl.rcParams['figure.dpi'] = 300 # DPI padrão para impressão, melhora resolução da imagem

sns.set_palette("husl")
sns.set_style("whitegrid")

Caso `.data_parquet` exista, leia todos os parquets.

_Parquet_ é um formato que guarda os dados com os devidos tipos, ao invés de apenas armazená-los em formato de texto.

---

Caso esses arquivos não existam, mostre uma aviso para realizar a preparação dos dados. Esse script pode ser executado a partir do arquivo `prepare_data.py` e necessita que uma chave de API do Kaggle seja disponibilizada em formato JSON (`kaggle.json`) na raíz do repositório. Esse arquivo se assemelha com algo do tipo:
```json
{"username":"nome_usuario","key":"abcdefghijklmnopqrstuvwxyz12345"}
```

In [ ]:
raw_dfs: dict[str, pd.DataFrame] = {}
if os.path.exists(".data_parquet"):
    for filename in os.listdir(".data_parquet"):
        if filename.endswith(".parquet"):
            treated_filename = filename.replace(".parquet", "")
            raw_dfs[treated_filename] = pd.read_parquet(f".data_parquet/{filename}")
else:
    print("Please, run the prepare_data.py script first.")

# Visualizações

## Construtores

Popularmente chamada de "equipe", os construtores dizem respeito aos responsáveis por construírem os carros e inscreverem pilotos em uma determinada corrida.

### Distribuição de nacionalidades

In [ ]:
fig, ax = plt.subplots(figsize=(24, 13))
# SNS count plot order by value counts
sns.countplot(raw_dfs["constructors"], x="nationality", order=raw_dfs["constructors"]["nationality"].value_counts().index)
# Create folder if not exists
if not os.path.exists(".output"):
    os.makedirs(".output")
ax.set_title("Distribuição de construtoras por nacionalidade")
ax.set_ylabel("Quantidade de construtoras")
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, horizontalalignment='center')
ax.set_xlabel("Nacionalidade")
plt.savefig(".output/1_distribuicao_construtoras_nacionalidade")
plt.show()

Nota-se:

- Há um domínio de equipes britânicas e americanas no esporte em toda sua história.
  - Isso faz sentido levando em conta o poderio econômico desses países, além do fato da Grã-Bretanha ser até hoje o país que mais consume automobilismo.
- Pelas mesmas razões, outros países europeus menores apresentam uma alta quantidade de construtores também.
- Nota-se que há muitos países que contém apenas uma equipe. Há casos como o Brasil, com equipes como a Copersucar, e casos de "equipes fantasma", que são abordadas na seção a seguir.

### Filtrando construtores "fantasma"

Por questões de curiosidade, ao investigar qual era a equipe da Rodésia do Sul, descobri que se trata de uma construtora que se inscreveu para apenas uma prova na década de 60, mais especificamente, o GP da África do Sul de 1965. Apesar da inscrição ter sido feita, a equipe não chegou nem a participar do Grande Prêmio. (https://www.statsf1.com/en/realpha.aspx)

Para filtrar os construtores apresentados e exigir um pouco mais de consistência dos nossos dados, iremos filtrar todos os construtores que realizaram ao menos 5 voltas em toda sua história. Para isso, precisaremos usar o conjunto de dados de resultados de corridas, filtrando-os por `constructorId`.

In [ ]:
constructors_df = raw_dfs["constructors"].copy()
if "laps" in constructors_df.columns:
    constructors_df = constructors_df.drop("laps", axis=1)
constructors_df_laps = raw_dfs["results"][[
    "constructorId",
    "laps"
]].groupby("constructorId").sum()
constructors_df = constructors_df.join(constructors_df_laps)
constructors_df = constructors_df.fillna(0).astype({"laps": int})
constructors_df

<text style="color: red">Nota-se: foi usado agrupamento por `constructorId`, ignorando completamente o fato de que dois (ou mais, a depender da temporada) pilotos podem atuar por uma mesma equipe em determinada corrida. Para nossos propósitos de filtragem, isso é irrelevante.</text>

<a id='constructor_lap_distribution'></a>
Quais equipes com mais voltas-piloto na história da Fórmula 1? Para facilitar visualização, iremos limitar às 15 primeiras.

In [ ]:
fig, ax = plt.subplots(figsize=(16, 9))

constructors_df.sort_values("laps", ascending=False).head(15).plot(kind="bar", x="name", y="laps", ax=ax)
ax.set_title("15 construtoras com maior número de voltas-piloto")
ax.set_xlabel("Construtora")
ax.set_ylabel("Quantidade de voltas-piloto")
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig(".output/3_construtores_maior_numero_voltas_piloto")
plt.show()

Novamente, esse resultado faz todo sentido levando em conta que a Scuderia Ferrari foi a única equipe a participar de todas as temporadas da Fórmula 1 desde 1950.

Voltando ao tema principal dessa seção, queremos avaliar a distribuição de nacionalidades excluindo as equipes fantasma.

In [ ]:
fig, ax = plt.subplots(figsize=(16, 9))
LAPS = 0
filtered_constructors_df = constructors_df.query("laps > @@LAPS")
ignored_constructors_df = constructors_df.query("laps <= @@LAPS")
sns.countplot(
    filtered_constructors_df,
    x="nationality",
    order=filtered_constructors_df["nationality"].value_counts().index,
    ax=ax)
ax.set_title(f"Nationality distribution of Constructors with at least {LAPS + 1} lap")
ax.set_xlabel("Count")
ax.set_ylabel("Nationality")
plt.xticks(rotation=45)
plt.show()

### Equipes fantasma filtradas e o caso da Equipe Lotus

Para efeito de comparação, vamos verificar quantas equipes nunca completaram 20 voltas em toda sua história.

In [ ]:
from pandas.plotting import table
fig, ax = plt.subplots()

ax.xaxis.set_visible(False)
ax.yaxis.set_visible(False)
table(
    ax=ax,
    data=ignored_constructors_df.rename(columns={
        "constructorRef": "1",
        "name": "Nome",
        "nationality": "Nacionalidade",
        "url": "2",
        "laps": "3",
    }).drop(columns=["1", "2", "3"]),
    loc="center",
    colWidths=[0.2]*len(ignored_constructors_df.columns),
    # row heights
    cellLoc="center",
    colLoc="center",
)
ax.set_title("Construtoras fantasma")
plt.savefig(".output/2_construtoras_fantasma")
ignored_constructors_df

Um contexto histórico precisa ser levado em conta nessa análise. No passado, era comum algumas construtoras usar diferentes motores em algumas etapas, então, embora a equipe Lotus tenha tido um número relevante de voltas em toda sua história...

In [ ]:
lotus_df = constructors_df.query("name.str.contains('Lotus')", engine="python")
lotus_df

Essa iteração da equipe (dentre as outras listada acima) não realizou nenhuma volta.

A Equipe Lotus já pertenceu a diferentes grupos, e cada uma desses grupos irão render uma diferente "iteração" da equipe. Infelizmente, não há como fazer uma aglutinação dessas diferentes iterações em um só registro de forma trivial.

Espera-se também que isso tenha impactado o gráfico de distribuição de voltas-piloto por equipe. Para investigar, iremos aglutinar todas as iterações britânicas da equipe Lotus, somar a quantidade de voltas e qual seria o resultado dentre os outros construtores.

In [ ]:
total_laps = lotus_df["laps"].sum()
total_laps

**Esse resultado colocaria a Lotus como a quarta equipe com mais voltas-piloto na história da Fórmula 1, ultrapassando Red Bull, Renault, Sauber e Tyrrell.**

### [Distribuição de voltas](#constructor_lap_distribution)

### Distribuição de vitórias

In [ ]:
if "wins" in constructors_df.columns:
    constructors_df = constructors_df.drop("wins", axis=1)
constructors_df_wins = raw_dfs["results"][[
    "constructorId",
    "position"
]].query("position == 1").groupby("constructorId").count()
constructors_df_wins.columns = ["wins"]
constructors_df = constructors_df.join(constructors_df_wins)
constructors_df = constructors_df.fillna(0).astype({"wins": int})
constructors_df

In [ ]:
fig, ax = plt.subplots(figsize=(16, 9))

constructors_df.sort_values("wins", ascending=False).head(15).plot(kind="bar", x="name", y="wins", ax=ax)
ax.set_title("15 construtoras com maior número de vitórias")
ax.set_xlabel("Construtora")
ax.set_ylabel("Número de vitórias")
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig(".output/4_construtores_maior_numero_vitorias")
plt.show()

# Análise de resultados

Iremos fazer uma análise sobre o _DataFrame_ contendo as posições de largada e finalização, analisando a distribuição de posições de chegada para cada posição de largada, além de analisar os motivos de não-finalização e abandono (a diferença será discutida mais à frente)

## Distribuição de causas de não-finalização

Uma não-finalização é indicada por uma posição de chamada NaN (_not a number_)

In [ ]:
dnf_df = raw_dfs["results"][[
    "grid",
    "position",
    "statusId",
]].copy()
dnf_df["grid"] = pd.to_numeric(dnf_df["grid"], "coerce")
dnf_df["position"] = pd.to_numeric(dnf_df["position"], "coerce")
dnf_df = dnf_df[dnf_df["position"].isnull()]
dnf_df = dnf_df.join(raw_dfs["status"], on="statusId")
dnf_df.drop(columns=["statusId"], inplace=True)
dnf_df

Histograma das principais causas de não-finalização

In [ ]:
fig, ax = plt.subplots(figsize=(16, 9))

dnf_df["status"].value_counts().head(15).plot(kind="bar", ax=ax)
plt.xticks(rotation=45)
ax.set_title("15 motivos mais comuns de não finalização")
ax.set_xlabel("Motivo")
ax.set_ylabel("Quantidade")
plt.tight_layout()
plt.savefig(".output/5_motivos_nao_finalizacao")

Iremos fazer uma distinção entre não-finalizações em que o piloto chegou a largar (abandono) e não-finalizações em que o piloto nem sequer largou (desqualificação, falha em se qualificar, falha em pré-qualificar).

Ignorando pilotos que não haviam qualificado

In [ ]:
exclude_status = ["Did not qualify", "Did not prequalify", "Not classified", "Withdrew"]

fig, ax = plt.subplots(figsize=(16, 9))
dnf_df.query("status not in @@exclude_status", engine="python")["status"].value_counts().head(15).plot(kind="bar", ax=ax)
plt.xticks(rotation=45)
ax.set_title("15 motivos mais comuns de abandono")
ax.set_xlabel("Motivo")
ax.set_ylabel("Quantidade")
plt.tight_layout()
plt.savefig(".output/6_motivos_abandono")

Também é possível separar por piloto (p.e. abandonos do Lewis Hamilton)

In [ ]:
dnf_df.query("driverId == 1 & status not in @@exclude_status", engine="python")["status"].value_counts().head(15).plot(kind="bar")

Será que existe diferentes distribuições de abandono para cada posição de largada? A animação gerada pela célula abaixo salva uma animação que mostra a distribuição de abandonos para diferentes posições de largada.

In [ ]:
# It would be interesting to show how many DNFs we have in this. For now, only ignoring.
import numpy as np
import matplotlib.animation as animation
from matplotlib.ticker import MaxNLocator, PercentFormatter

fig, ax = plt.subplots(figsize=(16, 9))
x = np.unique(dnf_df['grid'].sort_values().values)

def update(frame):
    this_x = x[frame]
    subset = dnf_df.query("grid == @@this_x & status not in @@exclude_status", engine="python")["status"].value_counts()
    this_data = subset.head(15)
    total = subset.sum()
    percentage = (this_data / total) * 100
    ax.cla()
    percentage.plot(kind="bar", ax=ax)
    if this_x == 0:
        title = "Histograma para posição de largada 0 (largada do pitlane ou não-largada)"
    else:
        title = f"Histograma para posição de largada {this_x}"
    ax.set_title(title)
    ax.yaxis.set_major_locator(MaxNLocator(integer=True))
    ax.yaxis.set_major_formatter(PercentFormatter())
    plt.xticks(rotation=45)
    ax.set_xlabel("Motivo")
    ax.set_ylabel("Porcentagem")
    plt.tight_layout()

anim = animation.FuncAnimation(fig=fig, func=update, frames=len(x), interval=2500)
os.makedirs(".animations", exist_ok=True)
anim.save(".animations/1_causas_de_abandono_por_posicao_largada.mp4")

## Para quem concluiu: distribuição de resultados por grid x posição

Ignorando resultados obtidos a partir de largada do pitlane

In [ ]:
position_grid_df = raw_dfs["results"][[
    "grid",
    "position",
]].copy()
position_grid_df["grid"] = pd.to_numeric(position_grid_df["grid"], "coerce")
position_grid_df["position"] = pd.to_numeric(position_grid_df["position"], "coerce")
position_grid_df = position_grid_df.dropna()
position_grid_df["position"] = position_grid_df["position"].astype(int)
ignored_pitlane_df = position_grid_df.query("grid > 0")

fig, ax = plt.subplots(figsize=(16, 9))
plt.hist2d(ignored_pitlane_df["grid"], ignored_pitlane_df["position"], (10, 10))
plt.colorbar()
plt.title("Heatmap de grid de largada x posição final")
plt.xlabel("Grid de largada")
plt.ylabel("Posição final")
plt.tight_layout()
plt.savefig(".output/7_heatmap_grid_posicao_final")
plt.show()

Histograma em uma dimensão, um frame para cada posição de largada. Incluindo largadas do pitlane.

In [ ]:
import numpy as np
import matplotlib.animation as animation
from matplotlib.ticker import MaxNLocator, PercentFormatter

fig, ax = plt.subplots(figsize=(16,9))
x = np.unique(position_grid_df['grid'].sort_values().values)

def update(frame):
    if frame < 10:
        this_x = 0
    else:
        this_x = x[frame - 10]
    this_data = position_grid_df[position_grid_df["grid"] == this_x]["position"].values
    ax.cla()
    sns.histplot(this_data, bins=33, binrange=(1, 33), ax=ax, stat='percent')
    if this_x == 0:
        title = "Histograma para posição de largada 0 (pitlane)"
    else:
        title = f"Histograma para posição de largada {this_x}"
    ax.set_title(title)
    ax.yaxis.set_major_locator(MaxNLocator(integer=True))
    ax.yaxis.set_major_formatter(PercentFormatter())
    ax.set_xlabel("Posição final")
    ax.set_ylabel("Porcentagem")
    plt.xticks(rotation=45)
    plt.tight_layout()
anim = animation.FuncAnimation(fig=fig, func=update, frames=len(x) + 10, interval=500)
anim.save(".animations/2_histograma_posicao_largada_chegada.mp4")

## Circuitos

Assim como a distribuição de nacionalidades dos construtores, é interessante analisar a distribuição de países cujos circuitos já foram utilizados para uma corrida de Fórmula 1.

In [ ]:
fig, ax = plt.subplots(figsize=(16, 9))
(
    raw_dfs["circuits"]
    .groupby("country")
    .size()
    .sort_values(ascending=False)
    .head(15)
    .plot(kind="bar", ax=ax)
)
ax.set_ylabel("Quantidade")
ax.set_xlabel("País")
ax.set_title("Top 15 países por número de circuitos")
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig(".output/8_top_15_paises_circuitos")
plt.show()

Podemos perceber uma notória dominância europeia e norte-americana. No entanto, note que estamos analisando a quantidade de circuitos, e não de corridas. Essa outra estatística é apresentada na próxima célula.

In [ ]:
fig, ax = plt.subplots(figsize=(16, 9))
(
    raw_dfs["races"]
    .join(raw_dfs["circuits"], on="circuitId", lsuffix="_race", rsuffix="_circuit")
    .groupby("country")
    .size()
    .sort_values(ascending=False)
    .head(15)
    .plot(kind="bar", ax=ax)
)
ax.set_ylabel("Quantidade de corridas")
ax.set_xlabel("País")
ax.set_title("15 países com maior número de corridas")
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig(".output/9_top_15_paises_corridas")
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(16, 9))
(
    raw_dfs["races"]
    .join(raw_dfs["circuits"], on="circuitId", lsuffix="_race", rsuffix="_circuit")
    .groupby("name_circuit")
    .size()
    .sort_values(ascending=False)
    .head(15)
    .plot(kind="bar", ax=ax)
)
ax.set_ylabel("Quantidade de corridas")
ax.set_xlabel("Circuito")
ax.set_title("15 circuitos com maior número de corridas")
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig(".output/10_top_15_circuitos_corridas")
plt.show()

In [ ]:
race_id_circuit_country = raw_dfs["races"].join(raw_dfs["circuits"], on="circuitId", lsuffix="_race", rsuffix="_circuit")[["country", "name_circuit"]]
top_10_countries = (
    race_id_circuit_country
    .groupby("country")
    .size()
    .sort_values(ascending=False)
    .head(10)
    .index
)

fig, ax = plt.subplots(figsize=(16,9))
color_palette = sns.color_palette("husl", n_colors=len(top_10_countries))

def update(frame):
    country = top_10_countries[frame]
    this_data = race_id_circuit_country.query("country == @country").groupby("name_circuit").size().sort_values(ascending=False)
    ax.cla()
    sns.barplot(
        this_data,
        order=this_data.index,
        ax=ax,
        color=color_palette[frame]
    )
    # Change color for each frame
    ax.set_title(f"Circuitos mais frequentes: {country}")
    ax.set_ylabel("Quantidade de corridas")
    ax.set_xlabel("Nome do circuito")
    plt.xticks(rotation=45)
    fig.tight_layout()

anim = animation.FuncAnimation(fig=fig, func=update, frames=len(top_10_countries), interval=5000)
anim.save(".animations/3_circuitos_por_pais.mp4")

### Nota sobre país sede e localização de circuitos

Estamos tratando circuitos puramente com base em sua localização geográfica. Alguns circuitos, embora localizados em determinado país, podem ter composto um Grande Prêmio com nome diferente. Um exemplo clássico é o Grande Prêmio de San Marino, cujo circuito (Autodromo Enzo e Dino Ferrari) está localizado na Itália, e não em San Marino. A Fórmula 1 usa esse tipo de nomenclatura para evitar conflitos com Grande Prêmios já existentes. Alguns outros casos são:
- GP da Europa (ocorridos em diferentes países, como Reino Unido, Alemanha, Espanha e até Azerbaijão)
- GP de Luxemburgo (realizado na Alemanha)
- GP da Suíça (realizado na França em 1982; a Suíça baniu o automobilismo em 1955 após um acidente nas 24h de Le Mans e só desbaniu o mesmo em 2022)


### Pontos importantes

- Apesar do domínio britânico do esporte, a Itália apresenta uma longa tradição de corridas, com o Autódromo de Monza tendo sediado corrida em todos anos com exceção de 1980. O Grande Prêmio de San Marino fez com que a Itália seja - com certa margem - o país que mais sediou corridas na história da Fórmula 1.
- Apesar de não ter sediado tantas corridas quanto potências europeias tal qual Itália, Alemanha e Reino Unido, os Estados Unidos apresentam uma grande quantidade de circuitos quando comparado com esses países, tendo 11 circuitos no total. Essa estatística desconsidera a temporada de 2024, onde o novo circuito de Las Vegas foi introduzido.
- A hegemonia europeia sobre o esporte fica ainda mais evidente quando analisando o número de corridas por país. Apenas 5 dos 15 países que mais sediaram corridas não são europeus: EUA, Canadá, Brasil, Japão e Austrália.

## Pilotos

Vamos combinar o primeiro nome e o sobrenome para facilitar visualização nos próximos gráficos

In [ ]:
drivers_df_name = raw_dfs["drivers"].copy()
drivers_df_name["name"] = drivers_df_name["forename"] + " " + drivers_df_name["surname"]
drivers_df_name = drivers_df_name.drop(columns=["forename", "surname"])
drivers_df_name

Distribuição por nacionalidade

In [ ]:
fig, ax = plt.subplots(figsize=(16, 9))
raw_dfs["drivers"]["nationality"].value_counts().head(15).plot(kind="bar", ax=ax)
ax.set_title("15 nacionalidades com mais pilotos")
ax.set_ylabel("Quantidade de pilotos")
ax.set_xlabel("Nacionalidade")
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig(".output/11_top_15_nacionalidades_pilotos")
plt.show()

A tendência de domínio europeu e norte-americano ainda é acentuado na distribuição de pilotos. Essa distribuição está muito próxima da distribuição de construtores e circuitos. Um dos pontos a se considerar é que, durante a década de 1950, as 500 milhas de Indianápolis faziam parte do calendário da Fórmula 1, o que provavelmente inflacionou a quantidade de pilotos estadunidenses contidos nesse conjunto de dados analisado.

Vamos analisar essa importância fazendo a remoção dos resultados da Indy 500 de 1950 até 1960.

In [ ]:
raceId_indy = (
    raw_dfs["races"]
    .query("circuitId == 19 & year >= 1950 & year <= 1960")
    .index
)
drivers_excluding_indy = (
    raw_dfs["results"]
    .query("raceId not in @raceId_indy")
    .index
    .get_level_values("driverId")
    .unique()
)
fig, ax = plt.subplots(figsize=(16, 9))
raw_dfs["drivers"]["nationality"].loc[drivers_excluding_indy].value_counts().head(15).plot(kind="bar", ax=ax)
ax.set_title("15 nacionalidades com mais pilotos (excluindo Indianápolis)")
ax.set_ylabel("Quantidade de pilotos")
ax.set_xlabel("Nacionalidade")
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig(".output/12_top_15_nacionalidades_pilotos_excluindo_indy")

Nota-se que a contribuição desses resultados para a distribuição total é muito relevante, ainda mais tendo em vista a alta quantidade de pilotos que participam da Indy 500. Tratam-se de 33 pilotos todo ano, com maioria estadunidense, e com uma alta rotatividade.

Distribuição de vencedores de corridas

In [ ]:
fig, ax = plt.subplots(figsize=(16, 9))
(
    raw_dfs["results"]
    .query("position == 1")
    .join(drivers_df_name, on="driverId")["name"]
    .value_counts()
    .head(15)
    .plot(kind="bar", ax=ax)
)
ax.set_title("15 pilotos com mais vitórias")
ax.set_ylabel("Quantidade de vitórias")
ax.set_xlabel("Piloto")
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig(".output/13_top_15_pilotos_vitorias")

Lewis Hamilton e Michael Schumacher apresentam os números mais expressivos da Fórmula 1 em termos de vitórias, sendo os únicos a se aproximarem (e no caso de Lewis Hamilton, atingir e ultrapassar) as 100 vitórias.

Distribuição de vencedores de corrida por nacionalidade

In [ ]:
fig, ax = plt.subplots(figsize=(16, 9))
winning_drivers_df = raw_dfs["results"].query("position == 1").join(drivers_df_name, on="driverId")
winning_drivers_df["nationality"].value_counts().head(15).plot(kind="bar", ax=ax)
ax.set_title("15 nacionalidades com mais vitórias")
ax.set_ylabel("Quantidade de vitórias")
ax.set_xlabel("Nacionalidade")
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig(".output/14_top_15_nacionalidades_vitorias")

Uma estatística interessante a se notar é como apesar do esporte ser completamente dominado por europeus (o que é esperado pela demografia observada anteriormente), o Brasil aparece como terceiro lugar na lista de nacionalidades por número de vitórias. É interessante verificar quais pilotos do Brasil mais contribuíram para essa estatística.

In [ ]:
fig, ax = plt.subplots(figsize=(16, 9))
(
    winning_drivers_df
    .query("nationality == 'Brazilian'")["name"]
    .value_counts()
    .head(15)
    .plot(kind="bar", ax=ax)
)
ax.set_title("Pilotos brasileiros com mais vitórias")
ax.set_ylabel("Quantidade de vitórias")
ax.set_xlabel("Piloto")
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig(".output/15_pilotos_brasileiros_vitorias")

Ayrton Senna foi o piloto brasileiro mais influente, tanto em números quanto em impacto, isso é refletido no histograma acima.

Vamos analisar a distribuição de _pole positions_

In [ ]:
fig, ax = plt.subplots(figsize=(16, 9))
(
    raw_dfs["results"]
    .query("grid == 1")
    .join(drivers_df_name, on="driverId")["name"]
    .value_counts()
    .head(15)
    .plot(kind="bar", ax=ax)
)
ax.set_title("15 pilotos com mais poles")
ax.set_ylabel("Quantidade de poles")
ax.set_xlabel("Piloto")
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig(".output/16_top_15_pilotos_poles")

Lewis Hamilton, com certa sobra, é o piloto da Fórmula 1 com os melhores números em qualificação, tendo ultrapassado as 100 pole positions. Nota-se que Ayrton Senna tem um número de poles muito superior ao seu número de vitórias.

In [ ]:
fig, ax = plt.subplots(figsize=(16, 9))
(
    raw_dfs["results"]
    .query("grid == 1")
    .join(drivers_df_name, on="driverId")["nationality"]
    .value_counts()
    .head(15)
    .plot(kind="bar", ax=ax)
)
ax.set_title("15 nacionalidades com mais poles")
ax.set_ylabel("Quantidade de poles")
ax.set_xlabel("Nacionalidade")
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig(".output/17_top_15_nacionalidades_poles")

Como esperado por conta da altíssima correlação entre poles e vitórias, a distribuição de nacionalidades por número de poles se aproxima muito da distribuição de nacionalidades por número de vitórias.

Um fenômeno que pode ser observado é a falta de conquistas muito acentuadas por parte de pilotos estadunidenses. Apesar de consistirem demografia consistente na história da categoria, pilotos dos EUA não possuem tantas vitórias ou pole positions.

# Conclusão

Podemos observar que a Fórmula 1 é uma categoria altamente concentrada na Europa, mais especificamente, Grã-Bretanha, França e Itália, e Estados Unidos. Esses países aparecem na maioria das distribuições de nacionalidades estando próximo do topo.

Apesar da alta presença estadunidense no esporte, o número de vitórias/poles não acompanha a porcentagem da demografia.

O Brasil é um país com alta tradição na categoria, embora as conquistas estejam concentradas em um número pequeno de pilotos, diferente dos casos da Grã-Bretanha, França e Itália.

A Scuderia Ferrari é, com certa vantagem, a equipe mais tradicional da categoria, sendo a maior vencedora e participadora.

A Grã-Bretanha faz jus ao título de catedral do automobilismo, tendo visto a alta quantidade de pilotos vencedores, construtoras, circuitos e corridas.

Lewis Hamilton, no momento de construção desse conjunto de dados, era o piloto com os maiores números de vitória, pódios e pole positions. Isso parece mudar em breve com o domínio recente de Max Verstappen e da Red Bull Racing até agora na temporada de 2024.